In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('21~23년 데이터.csv')
df['Region'].fillna('NA', inplace=True)

In [2]:
search_teams = ['JD Gaming', 'Gen.G eSports', 'LNG Esports', 'T1', 'Bilibili Gaming', 'Weibo Gaming', 'KT Rolster', 'Dplus KIA', 'G2 Esports', 'Fnatic', 'MAD Lions', 'Cloud9', 'NRG', 'Team Liquid', 'Team BDS', 'GAM Esports',
               'PSG Talon', 'DWG KIA','Galatasaray Esports','DRX','Royal Never Give Up','Edward Gaming','Top Esports','100 Thieves']

In [3]:
win=df[df.Name.isin(search_teams)]
win['result'] = 1
lose=df[~df.index.isin(win.index)]
lose['result'] = 0

C:\Users\LG\AppData\Local\Temp\ipykernel_14464\2743380165.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  win['result'] = 1
C:\Users\LG\AppData\Local\Temp\ipykernel_14464\2743380165.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lose['result'] = 0


In [191]:
ndf = pd.concat([win, lose])

### 데이터 증폭 

In [192]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [193]:
ndf.iloc[:,4:]

,Win rate,K:D,GPM,GDM,Game duration,Kills / game,Deaths / game,Towers killed,Towers lost,FB%,...,GD@15,PPG,NASHPG,NASH%,CSM,DPM,WPM,VWPM,WCPM,result
0,64.10%,1.21,1903,141,31:16:00,15.7,12.9,7.1,4.4,58.5,...,1030,5.96,0.88,62.2,32.8,2340,3.5,1.27,1.72,1
1,60.60%,1.09,1868,109,31:20:00,12.7,11.6,7.0,4.6,49.6,...,843,5.23,0.89,64.2,33.4,2216,3.3,1.4,1.63,1
3,71.80%,1.49,1916,207,31:27:00,14.1,9.4,7.5,4.3,61.3,...,1077,5.08,0.88,66.1,34.6,2220,3.1,1.21,1.88,1
4,69.50%,1.52,1878,144,31:05:00,13.0,8.5,7.2,4.4,55.1,...,512,4.24,0.81,65.9,34.4,2022,3.2,1.36,1.7,1
7,55.30%,1.00,1827,26,31:07:00,12.8,12.7,6.4,5.6,46.5,...,-,-,0.75,54.2,32.8,2280,-,-,-,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,0.00%,0.42,1565,-286,34:43:00,7.0,16.5,2.3,8.8,75,...,-2579,-,0.25,16.7,30.9,1295,3.4,1.14,1.29,0
920,25.00%,0.62,1764,-273,25:56:00,12.3,19.8,3.5,8.0,25,...,-3056,-,0.25,33.3,31.6,1805,2.8,1.18,1.15,0
921,75.00%,1.74,1868,212,28:14:00,16.5,9.3,6.0,4.0,25,...,347,-,0.75,62.5,32.2,1922,3.1,1.23,1.45,0
922,33.30%,0.78,1632,-122,31:47:00,10.7,13.7,3.0,6.7,33.3,...,-1306,-,0.33,33.3,31.1,1280,2.5,0.92,1.19,0


In [194]:
ndf['Win rate'] = ndf['Win rate'].str.replace('%',"").astype(float)
ndf['Game duration'] = ndf['Game duration'].apply(lambda x: x.split(':')[0])
ndf['Game duration'] =ndf['Game duration'].astype(int)
ndf.replace('-', np.nan,inplace=True)
ndf.dropna(inplace=True)

In [195]:
nndf=ndf.iloc[:,4:]
nndf=nndf.astype(float)

In [196]:
# train, test 셋 분리
X = nndf.iloc[:,:-1]
y = nndf.iloc[:,-1]

In [197]:
smote = SMOTE( random_state=42)  # 1000개로 증가시키려면 sampling_strategy=1000로 설정

# SMOTE를 사용하여 데이터 증강
X_resampled, y_resampled = smote.fit_resample(X, y)

### 데이터 정규화

In [206]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaled_data = scaler.fit_transform(X_resampled)

In [247]:
X_smaple = np.concatenate([scaled_data, scaled_data+np.random.randn(), scaled_data + np.random.ranf(), scaled_data-np.random.ranf(), scaled_data+np.random.rand(),scaled_data-np.random.rand()])
y_sample = pd.concat([y_resampled, y_resampled, y_resampled, y_resampled,y_resampled,y_resampled])
y_sample.reset_index(drop=True, inplace=True)

In [248]:
finalDF = pd.DataFrame(X_smaple, columns=nndf.columns[:-1])
finalDF['result'] = y_sample

In [250]:
finalDF.to_csv('트레인데이터_정규화.csv', index=False, encoding='utf-8')